In [69]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matsim
import geojson
import yaml

# Path

In [86]:
root_path = r'../../../../data/intermediate/test/freightEmissions/'
iter_kw = "21"
basic_scenario_outputs = 'scenarioBasic/iter'+iter_kw+'/outputs/'
van_scenario_outputs = 'scenarioVan/iter'+iter_kw+'/outputs/'
cargo_bike_scenario_outputs = 'scenarioCB/iter' + iter_kw +'/outputs/'
scenario_kw = basic_scenario_outputs
output_path = r'../../../../data/intermediate/test/freightEmissions/' + scenario_kw+ '/simWrapperData/'

# Read events

In [47]:
emission_events = matsim.event_reader(root_path+scenario_kw+'output_events.xml.gz', types='coldEmissionEvent,warmEmissionEvent')
emission_events

<generator object event_reader at 0x000002BA04284270>

# Convert to DataFrame

In [25]:
event_keys = set()
events_list = []
for emission_event in emission_events:
    # type(emission_event)
    # print(emission_event)
    events_list.append(emission_event)
    current_keys = list(emission_event.keys())
    event_keys.update(current_keys)
event_keys

{'BC_exhaust',
 'Benzene',
 'CH4',
 'CO',
 'CO2_TOTAL',
 'CO2_rep',
 'CO2e',
 'FC',
 'FC_MJ',
 'HC',
 'N2O',
 'NH3',
 'NMHC',
 'NO2',
 'NOx',
 'PM',
 'PM2_5',
 'PM_non_exhaust',
 'PN',
 'Pb',
 'SO2',
 'linkId',
 'time',
 'type',
 'vehicleId'}

In [26]:
events_dict = {}
for idx, event in enumerate(events_list):
    event_dict = {}
    for key in event_keys:
        if key in event.keys():
            event_dict[key] = event[key]
        else:
            event_dict[key] = None
    events_dict[idx] = event_dict
    

In [27]:
events_df = pd.DataFrame.from_dict(events_dict, orient='index')
events_df

,Benzene,Pb,NH3,NMHC,linkId,type,FC,PN,PM_non_exhaust,N2O,...,FC_MJ,CO2e,SO2,PM,HC,CO2_rep,CO2_TOTAL,CH4,vehicleId,NO2
0,0.288929879665375,0.0,None,4.0106520652771,27566523_11,coldEmissionEvent,52.3460388183594,0.0,None,None,...,2.21796584129333,193.72981262207,7.79413850978017E-4,0.0,4.24272966384888,154.470077514648,161.47184753418,0.232077300548553,freight_carrier_0_veh_carrier0_Van_0_1,0.0152425747364759
1,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
2,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
3,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
4,0.0525576248764992,0.0,None,0.729555428028107,27566523_11,coldEmissionEvent,3.80182957649231,0.0,None,None,...,0.161088198423386,14.7043743133545,5.66078924748581E-5,0.0,0.771771311759949,11.218976020813,11.7275066375732,0.0422158911824226,freight_carrier_0_veh_carrier0_Van_0_1,0.0108617367222905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,3.8280981384707365E-6,0.0,0.0014942834489833432,2.2968589500664639E-4,1012971580_0,warmEmissionEvent,3.3350304256272856,5.094064836795652E9,0.0014960097075195629,6.613628880535029E-5,...,0.1413093309859155,None,4.96574164590148E-5,2.8993235275231763E-5,3.828098071486719E-4,9.841479485557981,10.287570505778264,1.531239255388296E-4,freight_carrier_0_veh_carrier0_Van_0_1,9.805452191253604E-5
2943,1.4287446975839973E-6,0.0,5.577050214484201E-4,8.572468435505584E-5,27566523_r_7,warmEmissionEvent,1.2447191436946257,1.9012360345673814E9,5.583493055396498E-4,2.4683764242856134E-5,...,0.05274027730279731,None,1.8533425187965582E-5,1.0821021213902546E-5,1.428744672583839E-4,3.673093301884308,3.8395859456790085,5.714978790335994E-5,freight_carrier_0_veh_carrier0_Van_0_1,3.659646988899699E-5
2944,4.864895997052162E-7,0.0,1.898993522753423E-4,2.9189376833571974E-5,27566523_r_8,warmEmissionEvent,0.4238286371143767,6.473735713356447E8,1.9011873192390289E-4,8.404856799140648E-6,...,0.01795813936301347,None,6.310657611612847E-6,3.6845730994872915E-6,4.8648959119262675E-5,1.2506934885814707,1.3073844703716482,1.9459583988208663E-5,freight_carrier_0_veh_carrier0_Van_0_1,1.2461149999036437E-5
2945,9.386542757441719E-7,0.0,2.7730770612650424E-4,5.631926058467022E-5,27566523_r_9,warmEmissionEvent,0.6798415795019153,1.1991723568915517E9,3.7373007942753705E-4,1.2273509790133616E-5,...,0.028805723531020424,None,1.0122599432885608E-5,7.126424289808496E-6,9.386543099289556E-5,2.006172682227888,2.097107758784033,3.754617289439142E-5,freight_carrier_0_veh_carrier0_Van_0_1,1.9474489851411906E-5


In [28]:
# Convert columns which is number-like str into float and ignore the rest
for column in events_df.columns:
    try:
        events_df[column] = events_df[column].astype(float)
    except:
        pass

# Analysis

## Aggregate by link

In [29]:
link_co2e = events_df.pivot_table(index='linkId',
                                       columns='type', 
                                       values='CO2e', 
                                       aggfunc='sum')
link_co2e

type,coldEmissionEvent,warmEmissionEvent
linkId,,
100344071_8,14.704374,0.0
100490507_0,NaN,0.0
100490507_1,NaN,0.0
100490507_2,NaN,0.0
100490507_3-100490507_4,NaN,0.0
...,...,...
splitLink_123929657_r_1_2,NaN,0.0
splitLink_15216997_8_1,NaN,0.0
splitLink_15216997_8_2,NaN,0.0


In [30]:
link_co2e = link_co2e.fillna(0)
link_co2e['sum'] = link_co2e[['coldEmissionEvent', 'warmEmissionEvent']].sum(axis=1)
link_co2e 

type,coldEmissionEvent,warmEmissionEvent,sum
linkId,,,
100344071_8,14.704374,0.0,14.704374
100490507_0,0.000000,0.0,0.000000
100490507_1,0.000000,0.0,0.000000
100490507_2,0.000000,0.0,0.000000
100490507_3-100490507_4,0.000000,0.0,0.000000
...,...,...,...
splitLink_123929657_r_1_2,0.000000,0.0,0.000000
splitLink_15216997_8_1,0.000000,0.0,0.000000
splitLink_15216997_8_2,0.000000,0.0,0.000000


In [31]:
link_co2e['sum'].sum()

6108.871200561526

# Read move events

In [87]:
move_events = matsim.event_reader(root_path+scenario_kw+'output_events.xml.gz', types='entered link,left link')
move_events

<generator object event_reader at 0x000002BA06049C40>

In [88]:
event_keys = set()
events_list = []
for move_event in move_events:
    # type(emission_event)
    # print(emission_event)
    events_list.append(move_event)
    current_keys = list(move_event.keys())
    event_keys.update(current_keys)
event_keys

{'link', 'time', 'type', 'vehicle'}

In [89]:
events_dict = {}
for idx, event in enumerate(events_list):
    event_dict = {}
    for key in event_keys:
        if key in event.keys():
            event_dict[key] = event[key]
        else:
            event_dict[key] = None
    events_dict[idx] = event_dict

In [90]:
move_events_df = pd.DataFrame.from_dict(events_dict, orient='index')
move_events_df

,time,vehicle,type,link
0,107.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,27566523_11
1,107.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,27566523_r_9
2,109.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,27566523_r_9
3,109.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,27566523_r_10
4,113.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,27566523_r_10
...,...,...,...,...
5075,7100.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,27566523_r_8
5076,7101.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,27566523_r_8
5077,7101.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,27566523_r_9
5078,7103.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,27566523_r_9


In [83]:
move_events_df['vehicle'].unique()

array(['freight_carrier_0_veh_carrier0_Van_1_1'], dtype=object)

In [62]:
move_events_df.query('vehicle == "freight_carrier_0_veh_carrier0_CB_0_2"')

,time,vehicle,type,link
0,8.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,333784188_r_3
1,8.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,326736776_r_5
4,10.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,326736776_r_5
5,10.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,326736776_r_6
8,13.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,326736776_r_6
...,...,...,...,...
30919,42423.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,326736776_4
30920,42425.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,326736776_4
30921,42425.0,freight_carrier_0_veh_carrier0_CB_0_2,entered link,333784188_0
30922,42429.0,freight_carrier_0_veh_carrier0_CB_0_2,left link,333784188_0


In [65]:
def derive_vehicle_move_chain(move_events_df):
    vehicles_move_chain = {}
    for vehicle in move_events_df['vehicle'].unique():
        sub_df = move_events_df.query('vehicle == @vehicle')
        visited_links = []
        for idx, row in sub_df.iterrows():
            if row['link'] not in visited_links:
                visited_links.append(row['link'])
        vehicles_move_chain[vehicle] = visited_links
    return vehicles_move_chain

In [91]:
v_move_chains = derive_vehicle_move_chain(move_events_df)

In [75]:
def cal_vehicle_travel_distance(vehicle_move_chain: list, network: gpd.GeoDataFrame):
    total_distance = 0
    for idx, link_id in enumerate(vehicle_move_chain):
        link = network.query('link_id == @link_id')
        total_distance += link['length'].values[0]
    return total_distance


# Read network

In [32]:
network = matsim.read_network(
    root_path + 'diffusedGemeenteLeuvenWithHbefaType.xml.gz'
)
network

In [33]:
network_gdf = network.as_geo()
network_gdf

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
0,7.221720,1.543210,1030.864198,2.0,1,"bike,car",100006229_1-1085376891_0,453075429,453075431,"LINESTRING (175094.462 171579.032, 175087.718 ..."
1,7.924752,1.543210,1030.864198,2.0,1,"bike,car",100006230_1-1085376884_0,1472593946,1156261854,"LINESTRING (175094.956 171547.998, 175102.682 ..."
2,8.518773,1.543210,1030.864198,2.0,1,"bike,car",100006231_0-1085376890_0,1156261856,453075428,"LINESTRING (175114.241 171571.774, 175106.042 ..."
3,50.128561,10.030864,2200.617284,3.5,1,"bike,car",100010104_0,1156293852,1156293879,"LINESTRING (174065.630 177001.873, 174056.230 ..."
4,14.524426,10.030864,2200.617284,3.5,1,"bike,car",100010104_1,1156293879,1156293875,"LINESTRING (174056.230 177051.111, 174070.620 ..."
...,...,...,...,...,...,...,...,...,...,...
103748,183.145612,10.030864,2200.617284,3.5,1,car,splitLink_9904390_5_2,splitNode_9904390_5_1,19792038,"LINESTRING (175632.401 172452.198, 175505.340 ..."
103749,102.305105,10.030864,2200.617284,3.5,1,car,splitLink_9904410_0_1,80702464,splitNode_9904410_0_1,"LINESTRING (176002.946 172889.690, 176056.498 ..."
103750,102.305105,10.030864,2200.617284,3.5,1,car,splitLink_9904410_0_2,splitNode_9904410_0_1,100399122,"LINESTRING (176056.498 172976.855, 176110.050 ..."
103751,113.694402,1.543210,1030.864198,2.0,1,"bike,car",splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1,"LINESTRING (170932.314 173317.687, 171004.317 ..."


In [59]:
traveled_link = network_gdf[network_gdf['link_id'].isin(move_events_df['link'])]
traveled_link

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
116,6.635850,10.030864,2200.617284,3.5,1,"bike,car",100344071_8,1026364723,1026364672,"LINESTRING (172487.643 173655.320, 172489.985 ..."
118,66.534366,10.030864,2200.617284,3.5,1,"bike,car",100344071_r_0,1026364585,1026364672,"LINESTRING (172533.617 173598.885, 172489.985 ..."
119,6.635850,10.030864,2200.617284,3.5,1,"bike,car",100344071_r_1,1026364672,1026364723,"LINESTRING (172489.985 173649.111, 172487.643 ..."
299,11.881056,10.030864,1200.617284,6.0,1,"bike,car,pt",1003492531_0,1966766207,9260117550,"LINESTRING (174358.386 174526.533, 174356.220 ..."
300,11.881056,10.030864,1200.617284,6.0,1,"bike,car,pt",1003492531_r_0,9260117550,1966766207,"LINESTRING (174356.220 174538.214, 174358.386 ..."
...,...,...,...,...,...,...,...,...,...,...
103406,138.156503,10.030864,2200.617284,3.5,1,"car,pt",splitLink_44932919_0_3,splitNode_44932919_0_2,1041771651,"LINESTRING (173559.878 175430.121, 173438.515 ..."
103427,122.448924,10.030864,2200.617284,3.5,1,car,splitLink_55067676_0_1,9695872263,splitNode_55067676_0_1,"LINESTRING (172751.082 173480.888, 172665.405 ..."
103428,122.448924,10.030864,2200.617284,3.5,1,car,splitLink_55067676_0_2,splitNode_55067676_0_1,692826774,"LINESTRING (172665.405 173568.364, 172579.728 ..."
103429,122.448924,10.030864,2200.617284,3.5,1,car,splitLink_55067676_r_0_1,692826774,splitNode_55067676_r_0_1,"LINESTRING (172579.728 173655.840, 172665.405 ..."


In [92]:
for vehicle, move_chain in v_move_chains.items():
    print(vehicle, cal_vehicle_travel_distance(move_chain, network_gdf))

freight_carrier_0_veh_carrier0_Van_0_1 49546.34411185866


# Output 

In [41]:
link_co2e.to_csv(output_path+van_scenario_outputs+'link_co2e.csv', index=True, sep=';', encoding='utf-8-sig')